In [1]:
import os
from dotenv import load_dotenv
load_dotenv()


GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")


os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [5]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="llama3-groq-8b-8192-tool-use-preview")
llm.invoke("hi").content

'Hello! How can I assist you today?'

In [4]:
message=[{"role":"system","content":"you are a helpful assistant"},
         {"role":"user","content":"hi how are you?"}]

In [6]:
result = llm.invoke(message)
print(result.content)

I'm doing well, thank you! How can I assist you today?


In [8]:
class Chatbot:
    def __init__(self,system=""):
        self.system=system
        self.message=[]
        if self.system:
            self.message.append({"role":"system","content":system})
    def __call__(self,message):
        self.message.append({"role":"user","content":message})
        result=self.execute()
        self.message.append({"role":"assistant","content":result})
        return result
        
    def execute(self):
        llm = ChatGroq(model_name="llama3-groq-8b-8192-tool-use-preview")
        result = llm.invoke(self.message)
        return result.content

In [9]:
bot=Chatbot(system="you are helpful assistant")


In [11]:
bot("hi how are you?")


"I'm doing well, thank you! How can I assist you today?"

In [12]:
bot.message


[{'role': 'system', 'content': 'you are helpful assistant'},
 {'role': 'user', 'content': 'hi how are you?'},
 {'role': 'assistant',
  'content': "I'm doing well, thank you! How can I assist you today?"}]

In [25]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.


Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
you have to answer "hi","hello","i am good".
""".strip()

### Pattern Breakdown:

^: This matches the start of a string. It means the string must begin with what follows.
Action:: This is a literal match. It means the string must have the text "Action:" at the beginning.
(\w+):

The parentheses () define a capture group. This allows you to extract part of the string that matches this section.

\w+ matches one or more word characters (letters, digits, and underscores). This will capture a word that follows "Action:".

:: This is a literal colon that separates the word matched by (\w+) from the rest of the string.
(.*):

This is another capture group, where .* matches any character (.) zero or more times (*), which means it captures everything that comes after the second colon.
What does it do?

This regex is looking for a string that:

Starts with the text "Action:".
Has a word right after it, separated by a colon.
Then, after another colon, it captures everything that follow

In [13]:
import re
action_re = re.compile('^Action: (\w+): (.*)')

In [14]:
# Sample strings
text1 = "Action: Move: North"
text2 = "Action: Jump: High"
text3 = "Error: Not an Action"

In [15]:
match1 = action_re.match(text1)


In [16]:
match1.group(1)

'Move'

In [17]:
match1.group(2)

'North'

In [18]:
import httpx
def wikipedia(query):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [19]:
import httpx
def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    return response.json()[0]["text"]

In [20]:
def calculate(number):
    return eval(number)

In [21]:
calculate("2+2")


4

In [22]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [23]:
def query(question,max_turns=5):
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result

In [26]:
prompt


'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop your output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\n\nYour available actions are:\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point\nsyntax if necessary\n\nwikipedia:\ne.g. wikipedia: Django\nReturns a summary from searching Wikipedia\n\nsimon_blog_search:\ne.g. simon_blog_search: Python\nSearch Simon\'s blog for that term\n\nExample session:\nQuestion: What is the capital of France?\nThought: I should look up France on Wikipedia\nAction: wikipedia: France\nPAUSE\n\nYou will be called again with this:\nObservation: France is a country. The capital is Paris.\n\nYou then output:\nAnswer: The capital of France is Paris\n\nPlease Note: if you get 

In [27]:
bot = Chatbot(prompt)


In [28]:
bot("hi how are you?")


'hi i am good'

In [29]:
bot("what is your job?")


"I'm sorry, but I don't have a job in the classical sense. I exist to assist with tasks and answer questions to the best of my abilities."

In [30]:
next_prompt="Tell me about quantum computing"


In [31]:
result=bot(next_prompt)


In [32]:
result

'Thought: I should look up quantum computing on Wikipedia\nAction: wikipedia: quantum computing\nPAUSE'

In [33]:
actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]


In [34]:
actions


[<re.Match object; span=(0, 36), match='Action: wikipedia: quantum computing'>]

In [35]:
action, action_input = actions[0].groups()


In [36]:
print(action)
print(action_input)

wikipedia
quantum computing


In [37]:
query("Fifteen * twenty five")

Thought: I can calculate that for you
Action: calculate: 15 * 25
PAUSE
 -- running calculate 15 * 25
Observation: 375
Answer: The result of 15 * 25 is 375


'Answer: The result of 15 * 25 is 375'